In [15]:
import pandas as pd
import os
from nautilus_trader.persistence.wranglers import BarDataWrangler
from nautilus_trader.persistence.loaders import CSVBarDataLoader
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.model.data import BarType

In [16]:
data_path = "./data"
tickers = ['AUD/CAD','AUD/JPY','AUD/NZD','CHF/JPY','EUR/GBP','EUR/USD','GBP/NZD','USD/JPY']

In [17]:
# catalog

CATALOG_PATH = os.getcwd() + "/catalog"

# create catalog directory if it doesn't exist
if not os.path.exists(CATALOG_PATH):
  os.mkdir(CATALOG_PATH)

# Create a catalog instance
catalog = ParquetDataCatalog(CATALOG_PATH)


In [19]:
# load all tickers into catalog

for ticker in tickers:
  instrument = TestInstrumentProvider.default_fx_ccy(ticker)
  data = os.path.join(data_path, ticker.replace('/','') + ".csv")
  df = CSVBarDataLoader.load(data)
  bar_type = BarType.from_str(ticker + ".SIM-1-DAY-LAST-EXTERNAL")
  wrangler = BarDataWrangler(
    bar_type = bar_type,
    instrument = instrument
    )
  bars = wrangler.process(df)
  catalog.write_data([instrument])
  catalog.write_data(bars)